In [1]:
from cellpose import models, io
from cellpose.io import *
from collections import defaultdict
import geopandas
import glob
import imagej
from jpype import JArray, JInt
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
import os
import re
import pandas
from pandas import DataFrame
from pathlib import Path
import scyjava
import seaborn
import shutil
import tkinter as tk
from tkinter import filedialog
from PIL import Image
import sys
import os

In [2]:
scyjava.config.add_option('-Xmx60g')
#ij = imagej.init('/home/saka/fiji-linux64/Fiji.app', mode = 'interactive')
ij = imagej.init('/Users/volkenlab/Documents/Jacques_Data/Fiji.app/', mode='interactive')
ij.ui().showUI()
ij.getVersion()

'2.14.0/1.54f'

X
Y
C
Z
T


## things to hide for now

In [3]:
showPolygonRoi = scyjava.jimport('ij.gui.PolygonRoi')
Overlay = scyjava.jimport('ij.gui.Overlay')
Regions = scyjava.jimport('net.imglib2.roi.Regions')
LabelRegions = scyjava.jimport('net.imglib2.roi.labeling.LabelRegions')
ZProjector = scyjava.jimport('ij.plugin.ZProjector')()
Duplicator = scyjava.jimport('ij.plugin.Duplicator')()
TrackMate = scyjava.jimport('fiji.plugin.trackmate.TrackMate')
ov = Overlay()
Model =  scyjava.jimport('fiji.plugin.trackmate.Model')
Settings= scyjava.jimport('fiji.plugin.trackmate.Settings')
Settings= scyjava.jimport('fiji.plugin.trackmate.Settings')
TrackMate = scyjava.jimport('fiji.plugin.trackmate.TrackMate')
Logger= scyjava.jimport('fiji.plugin.trackmate.Logger')
DetectorKeys= scyjava.jimport('fiji.plugin.trackmate.detection.DetectorKeys') 
ExportTracksToXML= scyjava.jimport('fiji.plugin.trackmate.action.ExportTracksToXML') 
TmXmlWriter= scyjava.jimport('fiji.plugin.trackmate.io.TmXmlWriter')
LogRecorder = scyjava.jimport('fiji.plugin.trackmate.util.LogRecorder')
SparseLAPTrackerFactory= scyjava.jimport('fiji.plugin.trackmate.tracking.jaqaman.SparseLAPTrackerFactory')
TMUtils = scyjava.jimport('fiji.plugin.trackmate.util.TMUtils')
HyperStackDisplayer = scyjava.jimport('fiji.plugin.trackmate.visualization.hyperstack.HyperStackDisplayer')
SelectionModel = scyjava.jimport('fiji.plugin.trackmate.SelectionModel')
CellposeDetectorFactory = scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeDetectorFactory')
FeatureFilter = scyjava.jimport('fiji.plugin.trackmate.features.FeatureFilter')
DisplaySetting = scyjava.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettings')
DisplaySettingsIO = scyjava.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettingsIO')
CaptureOverlayAction = scyjava.jimport('fiji.plugin.trackmate.action.CaptureOverlayAction')
PretrainedModel= scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeSettings.PretrainedModel')
ThresholdDetectorFactory= scyjava.jimport('fiji.plugin.trackmate.detection.ThresholdDetectorFactory')
TrackScheme = scyjava.jimport('fiji.plugin.trackmate.visualization.trackscheme.TrackScheme')
TrackTableView = scyjava.jimport('fiji.plugin.trackmate.visualization.table.TrackTableView')
AllSpotsTableView = scyjava.jimport('fiji.plugin.trackmate.visualization.table.AllSpotsTableView')
PretrainedModel = scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeSettings.PretrainedModel')
Logger= scyjava.jimport('fiji.plugin.trackmate.Logger')
DetectorKeys= scyjava.jimport('fiji.plugin.trackmate.detection.DetectorKeys') 
ExportTracksToXML= scyjava.jimport('fiji.plugin.trackmate.action.ExportTracksToXML') 
TmXmlWriter= scyjava.jimport('fiji.plugin.trackmate.io.TmXmlWriter')
LogRecorder = scyjava.jimport('fiji.plugin.trackmate.util.LogRecorder')
SparseLAPTrackerFactory= scyjava.jimport('fiji.plugin.trackmate.tracking.jaqaman.SparseLAPTrackerFactory')
TMUtils = scyjava.jimport('fiji.plugin.trackmate.util.TMUtils')
HyperStackDisplayer = scyjava.jimport('fiji.plugin.trackmate.visualization.hyperstack.HyperStackDisplayer')
SelectionModel = scyjava.jimport('fiji.plugin.trackmate.SelectionModel')
CellposeDetectorFactory = scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeDetectorFactory')
FeatureFilter = scyjava.jimport('fiji.plugin.trackmate.features.FeatureFilter')
DisplaySetting = scyjava.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettings')
DisplaySettingsIO = scyjava.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettingsIO')
CaptureOverlayAction = scyjava.jimport('fiji.plugin.trackmate.action.CaptureOverlayAction')
PretrainedModel= scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeSettings.PretrainedModel')
ThresholdDetectorFactory= scyjava.jimport('fiji.plugin.trackmate.detection.ThresholdDetectorFactory')
TrackScheme = scyjava.jimport('fiji.plugin.trackmate.visualization.trackscheme.TrackScheme')
TrackTableView = scyjava.jimport('fiji.plugin.trackmate.visualization.table.TrackTableView')
AllSpotsTableView = scyjava.jimport('fiji.plugin.trackmate.visualization.table.AllSpotsTableView')
PretrainedModel = scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeSettings.PretrainedModel')

rm = ij.RoiManager.getRoiManager()

## Cellpose to ROI

In [ ]:
base_image = Path(f"/home/saka/Documents/Lab_stuff/confocal/exp2/stack.tif").as_posix()
open_image = f'open("{base_image}")'
imp = ij.py.run_macro(open_image)
ij.ui().show(imp)

In [ ]:
data_info = {}
for element in range(len(imp.dims)):
    name = imp.dims[element]
    data_info[name] = imp.shape[element]
    print(name)
num_frame = data_info['T'] + 1

In [ ]:
for timepoint in range(1, num_frame):
    frame_number = timepoint - 1
    input_txt = Path(f"{directory_path}/outputs/Experiment-1647-Split Scenes-01_z2/cellpose/frame_{frame_number}_cp_outlines.txt")
    txt_fh = open(input_txt, 'r')
    for line in txt_fh:
            xy = line.rstrip().split(",")
            xy_coords = [int(element) for element in xy if element not in '']
            x_coords = [int(element) for element in xy[::2] if element not in '']
            y_coords = [int(element) for element in xy[1::2] if element not in '']
            xcoords_jint = JArray(JInt)(x_coords)
            ycoords_jint = JArray(JInt)(y_coords)
            polygon_roi_instance = scyjava.jimport('ij.gui.PolygonRoi')
            roi_instance = scyjava.jimport('ij.gui.Roi')
            imported_polygon = polygon_roi_instance(xcoords_jint, ycoords_jint, len(x_coords), int(roi_instance.POLYGON))
            imp.setRoi(imported_polygon)
            rm.addRoi(imported_polygon)
            roi_count = rm.getCount()
            rm.select(roi_count - 1)
            time_set = imp.setT(timepoint)   
            rm.runCommand('Update')
ij.py.run_macro("roiManager('Select All');")
f_name = os.path.basename(filepath)
f_name = os.path.splitext(f_name)[0]
rm.runCommand("Save", f"{directory_path}/" + f"{f_name}.zip")

# workflow 2.0

In [4]:
#define your parent directory
directory_path = '/Users/volkenlab/Documents/Anushka'

In [5]:
# open raw file
raw_image = ij.io().open("Z:/Jacques_Data/20230513 hp1lc3 ATG7KD LV/Scene 2/Experiment-1647-Split Scenes-02.czi")

[java.lang.Enum.toString] ZeissCZIReader initializing Z:\Jacques_Data\20230513 hp1lc3 ATG7KD LV\Scene 2\Experiment-1647-Split Scenes-02.czi
[java.lang.Enum.toString] [WARN] Unknown DetectorType value 'GaAsP-PMT' will be stored as "Other"
[java.lang.Enum.toString] [WARN] Unknown DetectorType value 'GaAsP-PMT' will be stored as "Other"
[java.lang.Enum.toString] [WARN] Unknown DetectorType value 'GaAsP-PMT' will be stored as "Other"
[java.lang.Enum.toString] [WARN] Unknown DetectorType value 'Multialkali-PMT' will be stored as "Other"


In [6]:
imp = ij.py.to_imageplus(raw_image)

### Images needed for single cell tracking
The T-stack is used for cell detection with trackmate. The 'max' z-projection is used to select cells that would need to be infected but not overloaded for our future single bacteria analysis. 

In [13]:
#creation of single slice T-stack
wanted_z = 2 # change this number based on your original cellpose training dataset
channels = raw_image[:, :, :, wanted_z, :]
final_image = ij.py.to_imageplus(channels)
final_image.setDimensions(4, 1, 121) # this is to make sure the dimensions are not mixed up
channels_image = directory_path + "/channels.tif"
saved = ij.IJ.save(final_image, ij.py.to_java(channels_image))

# creation of 'max' z-projection of raw image for future selection of cells of interest
method = 'max all'
zproject = ZProjector.run(imp, method)
zproject.setDimensions(4, 1, 121)
max_channels = directory_path + "/max_channels.tif"
ij.IJ.saveAs(zproject, "Tiff", ij.py.to_java(max_channels))

### single channel z stack isolation and saving

In [14]:
#raw_image = ij.io().open('/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/Experiment-1647-Split-Scenes-03.czi')
format = f'Tiff'

LC3_channel = 0
channel = raw_image[:, :, LC3_channel, :, :]
lc3 = ij.py.to_imageplus(channel)
lc3.setDimensions(1, 11, 121)
#ij.ui().show(lc3)
result_path = f"{directory_path}/channel_LC3.tif"
ij.IJ.saveAs(lc3, "Tiff", ij.py.to_java(result_path))

LV_channel = 2
channel = raw_image[:, :, LV_channel, :, :]
lv = ij.py.to_imageplus(channel)
lv.setDimensions(1, 11, 121)
result_path = f"{directory_path}/channel_LV.tif"
ij.IJ.saveAs(lv, "Tiff", ij.py.to_java(result_path))

bact_channel = 1
channel = raw_image[:, :, bact_channel, :, :]
bact = ij.py.to_imageplus(channel)
bact.setDimensions(1, 11, 121)
#ij.ui().show(bact)
result_path = f"{directory_path}/channel_bact.tif"
ij.IJ.saveAs(bact, "Tiff", ij.py.to_java(result_path))

### single cell tracking with Trackmate-Cellpose

In [ ]:
image = "/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/cell_channel.tif"
out = "/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/trackmate/"
if not os.path.exists(out):
    os.makedirs(out)
    
# Parameters for Detection
# Here, the user can specify parameters for the detection step in Trackmate (Threshold Detector)
dsettings = {
    'TARGET_CHANNEL' : ij.py.to_java(0),
    'SIMPLIFY_CONTOURS' : True,
    'CELLPOSE_MODEL' : PretrainedModel.CUSTOM,
    'CELLPOSE_MODEL_FILEPATH' : "/home/saka/Documents/CP_20220523_104016",
    'USE_GPU' : True,
    'OPTIONAL_CHANNEL_2' : ij.py.to_java(0),
    'CELL_DIAMETER' : 16.38,
    'CELLPOSE_PYTHON_FILEPATH' : "/home/saka/sw/local/fiji/2023/bin/python3.10",
}

# Parameters for Tracking
# Here, the user can specify parameters for the tracking step in Trackmate (LAP Tracker)
frame_gap = 2
tsettings = {
    'LINKING_MAX_DISTANCE' : 10.0,
    'ALLOW_GAP_CLOSING' : True,
    'GAP_CLOSING_MAX_DISTANCE' : 15.0,
    'MAX_FRAME_GAP' : ij.py.to_java(2),
    'ALLOW_TRACK_SPLITTING' : False,
    'SPLITTING_MAX_DISTANCE' : 15.0,
    'ALLOW_TRACK_MERGING' : False,
    'ALTERNATIVE_LINKING_COST_FACTOR' : 1.05,
    'MERGING_MAX_DISTANCE' : 15.0,
    'CUTOFF_PERCENTILE' : 0.9,
}

if (image[len(image)-4:] == ".tif"):

    # Open Image
    imp = ij.IJ.openImage(image)
    imp.show()
    ij.py.run_macro('run("Set Scale...", "distance=2048 known=202.83 unit=µm");')
            
    # Create Model
    model = Model()
    settings = Settings(imp)
        
    # Detector
    settings.detectorFactory = CellposeDetectorFactory()
    for parameter, value in dsettings.items():
        settings.detectorSettings[parameter] = value
        
    # Tracker
    settings.trackerFactory = SparseLAPTrackerFactory()
    settings.trackerSettings = settings.trackerFactory.getDefaultSettings()
    for parameter, value in tsettings.items():
        settings.trackerSettings[parameter] = value
        
    # Execute Tracking
    trackmate = TrackMate(model, settings)
    ok = trackmate.checkInput()
    if not ok:
        sys.exit(str(trackmate.getErrorMessage()))
    ok = trackmate.process()
    if not ok:
        sys.exit(str(trackmate.getErrorMessage()))
    selectionModel = SelectionModel(model)
        
    # Display
    ds = DisplaySettingsIO.readUserDefault()
    displayer = HyperStackDisplayer(model, selectionModel, imp, ds)
    displayer.render()
    displayer.refresh()
    trackscheme = TrackScheme(model, selectionModel, ds)
    trackscheme.render()
        
    # Save Data
    outFile = Path(out+"exportModel.xml")
    writer = TmXmlWriter(outFile)
    writer.appendModel(model)
    writer.appendSettings(settings)
    writer.writeToFile()
    csvFileTracks = Path(out+"exportTracks.csv")
    trackTableView = TrackTableView(model, selectionModel, ds)
    trackTableView.getSpotTable().exportToCsv(csvFileTracks)
    

# sort csv file from trackmate output

In [ ]:
df = pandas.read_csv("/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/trackmate/exportTracks.csv")
df.drop([0, 1, 2], axis=0, inplace=True)
df = df.reset_index(drop=True)

for index, row in df.iterrows():
    df.at[index, 'TRACK_ID'] = int(row['TRACK_ID'])
    df.at[index, 'FRAME'] = int(row['FRAME'])

df.sort_values(by=['TRACK_ID', 'FRAME'], inplace=True, ignore_index=True)
df.to_csv("/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/trackmate/exportTracks_edited.csv", index=False)

# obtain ROI set

In [ ]:
IJRoiExporter = scyjava.jimport('fiji.plugin.trackmate.action.IJRoiExporter')
model.setLogger(Logger.IJ_LOGGER)
logger = Logger.IJ_LOGGER
exporter = IJRoiExporter(imp, logger)
exporter.export(model.getSpots().iterable(True))
rm = ij.RoiManager.getRoiManager()
rm.save("/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/RoiSet_cells.zip");
rm.runCommand(imp,"Delete");

In [ ]:
from zipfile import ZipFile
with ZipFile("/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/RoiSet_cells.zip", 'r') as zip: 
    zip.extractall(path="/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/rois")

# create csv of tracks by row

In [ ]:
data = pandas.read_csv("/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/trackmate/exportTracks_edited.csv")
print(data)

import csv
file = open('/home/saka/Documents/Lab_stuff/confocal/exp1ATG7_scene3/trackmate/da_Tracks.csv', 'w', newline='')
csv_writer = csv.writer(file)

current_track = 0
first = True
string = ''
for index, row in data.iterrows():
    if int(row['TRACK_ID']) == current_track:
        if first:
            string = f'{index+1}'
            first = False
        else:
            string = string + ", " + f'{index+1}'
    else:
        csv_writer.writerow([string])
        print(string)
        print(current_track)

        string = f'{index+1}'
        current_track += 1
file.close()

# add ROIs to ROI Manager

In [12]:
df = pandas.read_csv("C:/Users/volkenlab/Documents/Anushka/exportTracks_edited.csv")
roiset = "C:/Users/volkenlab/Documents/Anushka/roiset cells/"
for index, row in df.iterrows():
    file = row['LABEL'] + '.roi'
    #imp2 = ij.IJ.openImage(roiset+file)
    rm.open(roiset+file)

# add color overlay for each track

In [ ]:
max_channels = directory_path + "/max_channels.tif"
imp = ij.IJ.openImage(max_channels)
imp.show()

In [10]:
data_info = {}
for element in range(len(imp.dims)):
    name = imp.dims[element]
    data_info[name] = imp.shape[element]
    print(name)
num_channel = data_info['C'] + 1

In [11]:
for n in range (1, num_channel):
    imp.setC(n)
    ij.IJ.resetMinAndMax(imp)
imp.setC(1)
ij.IJ.run(imp, "Green", "");
imp.setC(2)
ij.IJ.run(imp, "Red", "");

In [9]:
df2 = pandas.read_csv("C:/Users/volkenlab/Documents/Anushka/da_Tracks.csv", header=None)

rm = ij.RoiManager.getRoiManager()
    
import random
colors = ["black", "blue", "cyan", "green", "magenta", "orange", "red", "white", "yellow"]
    
for index, row in df2.iterrows():
    l = row[0].split(", ")
    random_color = random.choice(colors)
    for cell in l:
        cell_index = int(cell)
        roi = rm.select(cell_index)
        overlay_command = f"Overlay.addSelection('{random_color}', 5);"
        ij.py.run_macro(overlay_command)

### selection of single cells and quality control

In [ ]:
cell1 = [91, 3]
cell2 = [9]
cell3 = [10]

In [ ]:
# testing the merging of 2 tracks
part1 = df2.iloc[91]
part1 = part1[0].split(", ")
part2 = df2.iloc[3]
part2 = part2[0].split(", ")
final_part = part1 + part2
#print(final_part)

for roi in final_part: #just for quality control to overlay the cell with a single color
    cellid = int(roi)
    rm.select(cellid - 1)
    overlay_command = f"Overlay.addSelection('Red', 20);"
    ij.py.run_macro(overlay_command)

In [ ]:
cell = df2.iloc[10]
cell2 = cell[0].split(", ")
for roi in cell2: 
    cellid = int(roi)
    rm.select(cellid - 1)
    overlay_command = f"Overlay.addSelection('Green', 20);"
    ij.py.run_macro(overlay_command)

## Test Area

In [3]:
rm = ij.RoiManager.getRoiManager()
directory_path = '/Users/volkenlab/Documents/Anushka'

### Duplication of single cell signal 

In [15]:
# I don't know if it's because sometimes for heavy files when they are saved there is a warning message saying there are too heavy for normal opening
# but the classic command to open images doesn't work and it needs to be forced using imageJ macro language
bact_image = Path(f"{directory_path}/channel_bact.tif").as_posix()
#bact = ij.IJ.openImage(bact_image)
#ij.ui().show(bact)

lc3_image = Path(f"{directory_path}/channel_LC3.tif").as_posix()
#lc3 = ij.IJ.openImage(lc3_image)
#ij.ui().show(lc3)

lv_image = Path(f"{directory_path}/channel_LV.tif").as_posix()
#lv = ij.IJ.openImage(lv_image)
#ij.ui().show(lv)

In [30]:
# assumes that "add ROIs to ROI Manager" has already been run

cell_desired = 0 ## here the user can specify which row of Da_Tracks they want to obtain a single cell video for
signal_list = ["bact", "lc3", "lv"] ## here the user can specify which signal videos they want to isolate the cell from
for signal in signal_list:
    image_path = Path(f"{directory_path}/channel_{signal}.tif").as_posix()
    imp = ij.IJ.openImage(image_path)
    ij.ui().show(imp)
    #imp = ij.IJ.getImage()
    w = scyjava.jimport('ij.WindowManager')
    df = pandas.read_csv("C:/Users/volkenlab/Documents/Anushka/da_Tracks.csv", header=None)
    row = df.iloc[cell_desired]
    nums = row[0].split(", ")
    arr = []
    for x in nums:
        arr.append(int(x))
    rm = ij.RoiManager.getRoiManager()
    for cell in arr:
        roi = rm.getRoi(cell-1)
        roi2 = rm.select(cell-1)
        frame = roi.getTPosition()
        if frame == 1:
            ij.IJ.run("Duplicate...", "title=Stack_reg_image duplicate frames="+str(frame))
            ij.IJ.selectWindow(imp.getTitle())
        elif frame > 1:
            ij.IJ.run("Duplicate...", "title=channels duplicate frames="+str(frame))
            ij.IJ.run("Concatenate...", "title=Stack_reg_image open image1=Stack_reg_image image2=channels image3=[-- None --]")
            ij.IJ.selectWindow(imp.getTitle())
    new_vid =  w.getImage("Stack_reg_image")
    ij.IJ.save(new_vid, directory_path+"/"+signal+"_single_cell_"+str(cell_desired)+".tif")
    imp.close()
    w.getImage("Stack_reg_image").close()

In [ ]:
   
macro = """
title = getTitle();
a =
newArray(46, 103, 158, 211, 265, 320, 370, 417, 465, 510, 560, 
612, 670, 721, 776, 826, 878, 931, 979, 1030, 1082, 1132, 1181, 
1235, 1283, 1328, 1372, 1417, 1462, 1508, 1554, 1599, 1647, 1690, 
1735, 1776, 1822, 1871, 1920, 1965, 2007, 2052, 2098, 2145, 2186, 
2228, 2272, 2311, 2357, 2399, 2438, 2477, 2517, 2556, 2595, 2633, 
2669, 2710, 2751, 2791, 2824, 2858, 2891, 2928, 2965, 3002, 3044, 
3083, 3124, 3161, 3200, 3244, 3286, 3323, 3357, 3390, 3425, 3463, 3500, 3535, 3566, 3601, 3637
);
for ( i=0; i<a.length; i++ ) {
    roiManager( "Select", a[i]);
    Stack.getPosition(channel, slice, frame);
	run("Duplicate...", "duplicate frames="+frame);
    selectWindow(title);
}

selectWindow(title);
close();
run("Concatenate...", "all_open open");
"""

results = ij.py.run_macro(macro)

In [ ]:
macro = """
title = getTitle();
a =
newArray(22, 79, 134, 187, 243, 296, 350, 398, 443, 491, 540, 591, 646, 
700, 753, 805, 858, 911, 960, 1010, 1060, 1112, 1161, 1212, 1261, 1309, 
1353, 1396, 1442, 1487, 1534, 1581, 1630, 1672, 1718, 1757, 1801, 1851, 
1899, 1944, 1988, 2029, 2075, 2121, 2166, 2208, 2252, 2291, 2337, 2379, 
2418, 2460, 2501, 2540, 2579, 2616, 2654, 2694
);
for ( i=0; i<a.length; i++ ) {
    roiManager( "Select", a[i]);
    Stack.getPosition(channel, slice, frame);
	run("Duplicate...", "duplicate frames="+frame);
    selectWindow(title);
}

selectWindow(title);
close();
run("Concatenate...", "all_open open");
"""

results = ij.py.run_macro(macro)

In [ ]:
macro = """
title = getTitle();u
a =
newArray(48, 105, 159, 213, 266, 321, 372, 419, 466, 513, 561, 614, 671, 723, 777, 
828, 881, 933, 980, 1032, 1085, 1135, 1183, 1234, 1284, 1329, 1373, 1418, 1463, 1509,
1555, 1600, 1646, 1689, 1734, 1774, 1821, 1869, 1917, 1960, 2002, 2047, 2093, 2140, 
2181, 2223, 2267, 2307, 2353, 2394, 2434, 2476, 2519, 2557, 2596, 2634, 2670, 2711, 
2752, 2792, 2825, 2859, 2890, 2929, 2964, 3001, 3043, 3082, 3123, 3159, 3201, 3245, 
3289, 3326, 3360, 3394, 3430, 3469, 3504, 3538
);
for ( i=0; i<a.length; i++ ) {
    roiManager( "Select", a[i]);
    Stack.getPosition(channel, slice, frame);
	run("Duplicate...", "duplicate frames="+frame);
    selectWindow(title);
}

selectWindow(title);
close();
run("Concatenate...", "all_open open");
"""wanted_channel = 3
channel = image[:, :, wanted_channel, :, :]
imp = ij.py.to_imageplus(channel)
imp.setDimensions(1, 11, 121)

results = ij.py.run_macro(macro)

In [ ]:
macro = """
title = getTitle();
a =
newArray(17,74,132,185,240,295,347,397,440,488,536,586,645,
697,750,804,856,645,697,750,804,856,909,958,1008,1059,1111,
1160,1210,1260,1307,1352,1395,1441,1486,1533,1580,1628,1670,
1716,1756,1800,1849,1897,1942,1986,2027,2072,2119,2164,2205,
2248,2286,2334,2375,2412,2456,2498,2538,2576,2613,2652
);
for ( i=0; i<a.length; i++ ) {
    roiManager( "Select", a[i]);
    Stack.getPosition(channel, slice, frame);
	run("Duplicate...", "duplicate frames="+frame);
    selectWindow(title);
}

selectWindow(title);
close();
run("Concatenate...", "all_open open");
"""

results = ij.py.run_macro(macro)

In [ ]:
macro = """
title = getTitle();
a =
newArray(7, 64, 120, 171, 228, 281, 336, 387, 431, 
479, 526, 574, 629, 685, 737, 792, 844, 897, 947, 996, 
1045, 1100, 1149, 1197, 1248, 1296, 1341, 1384, 1431, 1478, 
1523, 1570, 1619, 1663, 1708, 1748, 1791, 1840, 1885, 1931, 
1976, 2017, 2061, 2108, 2153, 2193, 2237, 2277, 2322, 2365, 2404, 
2444, 2486, 2527, 2567, 2605, 2643, 2680, 2721, 2760, 2799, 2834, 
2866, 2899, 2936, 2974, 3013, 3050, 3093, 3131, 3169, 3210, 3254, 
3296, 3330, 3364, 3397, 3433, 3472, 3507, 3541, 3568, 3604, 3638, 
3673, 3703, 3741, 3773, 3808, 3843, 3877, 3907, 3939, 3969, 3996, 4028, 4057, 4085, 4113, 
4144, 4175, 4204, 4230, 4258, 4288, 4314, 4347, 4381, 4414, 4443, 4471, 4502, 4531, 4560, 4590, 4617, 4646, 4671, 4699, 4724, 4754
);
for ( i=0; i<a.length; i++ ) {
    roiManager( "Select", a[i]);
    Stack.getPosition(channel, slice, frame);
	run("Duplicate...", "duplicate frames="+frame);
    selectWindow(title);
}

selectWindow(title);
close();
run("Concatenate...", "all_open open");
"""

results = ij.py.run_macro(macro)

In [ ]:
macro = """
title = getTitle();
a =
newArray(27, 85, 138, 192, 247, 301, 355, 404, 448, 496, 544, 598, 655, 708, 761, 
815, 865, 916, 965, 1016, 1067, 1118, 1166, 1217, 1265, 1312, 1357, 1400, 1446, 1491, 
1538, 1586, 1635, 1679, 1725, 1764, 1811, 1860, 1907, 1951, 1995, 2037, 2083, 2129, 2174, 
2216, 2260, 2300, 2344, 2386, 2426, 2468, 2508, 2546, 2587, 2623, 2660, 2701, 2742, 2782, 2818, 
2849, 2883, 2918, 2955, 2993, 3031, 3073, 3115, 3152, 3188, 3233, 3277, 3316, 3346, 3379
);
for ( i=0; i<a.length; i++ ) {
    roiManager( "Select", a[i]);
    Stack.getPosition(channel, slice, frame);
	run("Duplicate...", "duplicate frames="+frame);
    selectWindow(title);
}

selectWindow(title);
close();
run("Concatenate...", "all_open open");
"""

results = ij.py.run_macro(macro)

In [ ]:
macro = """
title = getTitle();
a =
newArray(10, 67, 123, 175, 232, 286, 339, 390, 435, 482, 528, 580, 635, 690, 743, 
797, 851, 903, 952, 1002, 1052, 1104, 1154, 1205, 1256, 1303, 1348, 1392, 1437, 
1484, 1528, 1577, 1624, 1667, 1713, 1753, 1796, 1846, 1894, 1938, 1983, 2024, 2070, 
2116, 2160, 2200, 2245, 2284, 2330, 2373, 2411, 2454, 2495, 2537, 2575, 2612, 2651, 2690, 2730, 2771, 2806, 2841,
2875, 2909, 2947, 2983, 3024, 3064, 3106, 3144, 3180, 3221, 3267, 3309, 3341, 3374, 3410, 3442, 3481, 3516, 3549, 
3578, 3612, 3647, 3680, 3713, 3750, 3780, 3817, 3850, 3882, 3913, 3943, 3973, 4003, 4033, 4062, 4090, 4119, 4149, 
4180, 4209, 4239, 4262, 4293, 4322, 4356, 4387, 4422, 4450, 4479, 4510, 4538, 4567, 4596, 4624, 4652, 4677, 4704);
for ( i=0; i<a.length; i++ ) {
    roiManager( "Select", a[i]);
    Stack.getPosition(channel, slice, frame);
	run("Duplicate...", "duplicate frames="+frame);
    selectWindow(title);
}

selectWindow(title);
close();
run("Concatenate...", "all_open open");
"""

results = ij.py.run_macro(macro)

In [ ]:
macro = """
title = getTitle();
a =
newArray(3, 61, 113, 169, 226, 279, 334, 383, 429, 476, 522, 570, 625, 683, 735,
788, 840, 894, 946, 995, 1046, 1097, 1150, 1198, 1250, 1298, 1344, 1385, 1432, 
1476, 1522, 1568, 1617, 1660, 1706, 1745, 1790, 1838, 1887, 1936, 1980, 2020, 
2065, 2112, 2154, 2196, 2238, 2278, 2323, 2366, 2405, 2445, 2488, 2529, 2568, 
2606, 2644, 2682, 2722, 2763, 2800, 2832, 2867, 2900, 2939, 2975, 3014, 3054, 
3097, 3135, 3171, 3214, 3257, 3300, 3334, 3366, 3400, 3435, 3474, 3509, 3543, 3572, 3607, 3641, 3676, 3705, 3743, 3776, 3810
);
for ( i=0; i<a.length; i++ ) {
    roiManager( "Select", a[i]);
    Stack.getPosition(channel, slice, frame);
	run("Duplicate...", "duplicate frames="+frame);
    selectWindow(title);
}

selectWindow(title);
close();
run("Concatenate...", "all_open open");
"""

results = ij.py.run_macro(macro)

In [ ]:
import tkinter as tk
from tkinter import filedialog
root = tk.Tk()
root.withdraw()
directory_path = filedialog.askdirectory()

### Bacterial ROI collection and coordinates measurement

In [ ]:
rm = ij.RoiManager.getRoiManager()
file_pattern = os.path.join(directory_path, "*.tif")
file_list = glob.glob(file_pattern)
for file_path in file_list :
    image = ij.io().open(file_path)
    shown = ij.ui().show(image)
    roi_collection = """
    run("Duplicate...", "duplicate");
    run("Smooth", "stack");
    run("Smooth", "stack");
    setAutoThreshold("Default dark no-reset");
    run("Threshold...");
    setThreshold(5, 255);
    setOption("BlackBackground", true);
    run("Convert to Mask", "black");  
    run("Analyze Particles...", "size=1-Infinity add stack");
    run("Set Measurements...", "area centroid stack redirect=None decimal=2");
    nbArea=roiManager("count")
    for (i=0; i<nbArea; i++) {
		roiManager("Select", i);
		run("Measure");	
    }
    close();
    close();
    """
    rois = ij.py.run_macro(roi_collection)
    f_name = os.path.basename(file_path)
    f_name = os.path.splitext(f_name)[0]
    rm.runCommand("Select All")
    rm.runCommand("Save", f"{directory_path}/" + f"{f_name}.zip") # this saves the ROIs as a zip file
    rm.runCommand("Delete")
    measurements = ij.ResultsTable.getResultsTable() # call of the table
    measurements_table = ij.convert().convert(measurements, scyjava.jimport('org.scijava.table.Table')) # conversion to a java table object
    table = ij.py.from_java(measurements_table) # Conversion into a python dataframe from Java
    output_path = Path(f"{directory_path}/{f_name}.csv") # save giving a name matching the opened image
    table.to_csv(output_path)
    ij.py.run_macro(""" 
        title = Table.title();
        selectWindow(title);
        run("Close");
    """)    

### running trackmate on bacteria channel

In [ ]:
import sys
import os

Model =  scyjava.jimport('fiji.plugin.trackmate.Model')
Settings= scyjava.jimport('fiji.plugin.trackmate.Settings')
TrackMate = scyjava.jimport('fiji.plugin.trackmate.TrackMate')
Logger= scyjava.jimport('fiji.plugin.trackmate.Logger')
DetectorKeys= scyjava.jimport('fiji.plugin.trackmate.detection.DetectorKeys') 
ExportTracksToXML= scyjava.jimport('fiji.plugin.trackmate.action.ExportTracksToXML') 
TmXmlWriter= scyjava.jimport('fiji.plugin.trackmate.io.TmXmlWriter')
LogRecorder = scyjava.jimport('fiji.plugin.trackmate.util.LogRecorder')
SparseLAPTrackerFactory= scyjava.jimport('fiji.plugin.trackmate.tracking.jaqaman.SparseLAPTrackerFactory')
TMUtils = scyjava.jimport('fiji.plugin.trackmate.util.TMUtils')
HyperStackDisplayer = scyjava.jimport('fiji.plugin.trackmate.visualization.hyperstack.HyperStackDisplayer')
SelectionModel = scyjava.jimport('fiji.plugin.trackmate.SelectionModel')
CellposeDetectorFactory = scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeDetectorFactory')
FeatureFilter = scyjava.jimport('fiji.plugin.trackmate.features.FeatureFilter')
DisplaySetting = scyjava.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettings')
DisplaySettingsIO = scyjava.jimport('fiji.plugin.trackmate.gui.displaysettings.DisplaySettingsIO')
CaptureOverlayAction = scyjava.jimport('fiji.plugin.trackmate.action.CaptureOverlayAction')
PretrainedModel= scyjava.jimport('fiji.plugin.trackmate.cellpose.CellposeSettings.PretrainedModel')
ThresholdDetectorFactory= scyjava.jimport('fiji.plugin.trackmate.detection.ThresholdDetectorFactory')
TrackScheme = scyjava.jimport('fiji.plugin.trackmate.visualization.trackscheme.TrackScheme')
TrackTableView = scyjava.jimport('fiji.plugin.trackmate.visualization.table.TrackTableView')
AllSpotsTableView = scyjava.jimport('fiji.plugin.trackmate.visualization.table.AllSpotsTableView')

#reload(sys)
#sys.setdefaultencoding("utf-8")

# Directory
# Here, the user can specify the directory where the cell images are located
src = "/Users/volkenlab/Documents/Anushka/Images/"
out = src+"Output/"
if not os.path.exists(out):
    os.makedirs(out)

# Parameters for Detection
# Here, the user can specify parameters for the detection step in Trackmate (Threshold Detector)
dsettings = {
    'TARGET_CHANNEL' : ij.py.to_java(1),
    'SIMPLIFY_CONTOURS' : False,
    'INTENSITY_THRESHOLD' : 5.0,
}

# Parameters for Tracking
# Here, the user can specify parameters for the tracking step in Trackmate (LAP Tracker)
frame_gap = 2
tsettings = {
    'LINKING_MAX_DISTANCE' : 15.0,
    'ALLOW_GAP_CLOSING' : True,
    'GAP_CLOSING_MAX_DISTANCE' : 15.0,
    'MAX_FRAME_GAP' : ij.py.to_java(2),
    'ALLOW_TRACK_SPLITTING' : False,
    'SPLITTING_MAX_DISTANCE' : 15.0,
    'ALLOW_TRACK_MERGING' : False,
}

for image in os.listdir(src):
    if (image[len(image)-4:] == ".tif"):

        # Open Image
        imp = ij.IJ.openImage(src + image)
        imp.show()
        
        # Create Model
        model = Model()
        settings = Settings(imp)
        
        # Detector
        settings.detectorFactory = ThresholdDetectorFactory()
        for parameter, value in dsettings.items():
            #settings.detectorSettings.update({parameter:value})
            settings.detectorSettings[parameter] = value
        filter1 = FeatureFilter('QUALITY', 198, True)
        settings.addSpotFilter(filter1)
        print(settings.detectorSettings)
        
        # Tracker
        settings.trackerFactory = SparseLAPTrackerFactory()
        settings.trackerSettings = settings.trackerFactory.getDefaultSettings()
        for parameter, value in tsettings.items():
            #settings.trackerSettings.update({parameter:value})
            settings.trackerSettings[parameter] = value
        
        # Execute Tracking
        trackmate = TrackMate(model, settings)
        ok = trackmate.checkInput()
        if not ok:
            sys.exit(str(trackmate.getErrorMessage()))
        ok = trackmate.process()
        if not ok:
            sys.exit(str(trackmate.getErrorMessage()))
        selectionModel = SelectionModel(model)
        
        # Display
        ds = DisplaySettingsIO.readUserDefault()
        displayer = HyperStackDisplayer(model, selectionModel, imp, ds)
        displayer.render()
        displayer.refresh()
        trackscheme = TrackScheme(model, selectionModel, ds)
        trackscheme.render()
        
        # Save Data
        outFile = Path(out+image+"_exportModel.xml")
        writer = TmXmlWriter(outFile)
        writer.appendModel(model)
        writer.appendSettings(settings)
        writer.writeToFile()
        csvFileTracks = Path(out+image+"_exportTracks.csv")
        trackTableView = TrackTableView(model, selectionModel, ds)
        trackTableView.getSpotTable().exportToCsv(csvFileTracks)

### Joining of trackmate output and ROI coordinates from fiji

In [ ]:
from math import isnan


def xref_locations(first, second, first_x='POSITION_X', first_y='POSITION_Y', first_z='POSITION_Z',
                   second_x='X', second_y='Y', second_z='Slice',
                   max_dist=2, verbose=False):
    pairwise_elements = pandas.DataFrame()
    first_measurements = pandas.read_csv(first)
    first_measurements = first_measurements.drop([0,1,2])
    second_measurements = pandas.read_csv(second)
    first_gdf = geopandas.GeoDataFrame(
        first_measurements,
        geometry=geopandas.points_from_xy(first_measurements[first_x],
                                          first_measurements[first_y],
                                          first_measurements[first_z]))
    second_gdf = geopandas.GeoDataFrame(
        second_measurements,
        geometry=geopandas.points_from_xy(second_measurements[second_x],
                                          second_measurements[second_y],
                                          second_measurements[second_z]))
    ti_rows = first_gdf.shape[0]
    tj_rows = second_gdf.shape[0]
    for ti_row in range(0, ti_rows):
        if verbose:
            print(f"On row: {ti_row}")
        ti_element = first_gdf.iloc[[ti_row, ]]
        # Get the frame of the current element
        ti_frame = int(ti_element['FRAME'].values[0])  # Convert to integer values in 'FRAME' column
        # Filter second_gdf to only include elements with the same frame
        same_frame_elements = second_gdf[second_gdf['Frame'] == (ti_frame + 1)]
        
        titj = geopandas.sjoin_nearest(ti_element, same_frame_elements,
                                       distance_col="pairwise_dist",
                                       max_distance=max_dist)
        chosen_closest_dist = titj.pairwise_dist.min()
        if (isnan(chosen_closest_dist)):
            print(f"This element has no neighbor within {max_dist}.")
        else:
            chosen_closest_cell = titj.pairwise_dist == chosen_closest_dist
            chosen_closest_row = titj[chosen_closest_cell]
            pairwise_tmp = pandas.concat([pairwise_elements, chosen_closest_row])
            pairwise_elements = pairwise_tmp
    return pairwise_elements


In [ ]:
import matplotlib.pyplot as plt
file_pattern = os.path.join(directory_path, "*.tif")
file_list = glob.glob(file_pattern)

for i in range(1, len(file_list)):
    first = Path(f"{directory_path}/Output/cell_test_bact{i}.tif_exportTracks.csv")
    second = Path(f"{directory_path}/cell_test_bact{i}.csv")
    #first = '/home/saka/Documents/Lab_stuff/confocal/exp2/tracks_from_script.csv'
    #second = '/home/saka/Documents/Lab_stuff/confocal/exp2/results_bacteria.csv'
    pairwise = xref_locations(first, second, 
                          first_x='POSITION_X', 
                          first_y='POSITION_Y', 
                          first_z='POSITION_Z', 
                          second_x='X', 
                          second_y='Y', 
                          second_z='Slice', 
                          verbose=False)
    #pairwise.head()
    grouped = pairwise.groupby('TRACK_ID')['Unnamed: 0'].apply(list).reset_index()
    grouped.rename(columns={'Unnamed: 0': 'object_ID_list'}, inplace=True)
    final_csv = Path(f"{directory_path}/cell_test_bact{i}_grouped.csv")
    grouped.to_csv(final_csv)

In [ ]:
print(len(file_list))

### Quality control of bacterial tracking

In [ ]:
directory_path = '/home/saka/Documents/Lab_stuff/confocal/test_script_trackmate/'
rm = ij.RoiManager.getRoiManager()

In [ ]:
rm = ij.RoiManager.getRoiManager()
file_pattern = os.path.join(directory_path, "*.tif")
file_list = glob.glob(file_pattern)
colors = ["blue", "cyan", "green", "magenta", "orange", "red", "yellow"]
for file_path in file_list: # cycle through images, open them and open the matching "grouped' csv file
    imp = ij.IJ.openImage(file_path)
    ij.ui().show(imp)
    f_name = os.path.basename(file_path)
    f_name = os.path.splitext(f_name)[0]
    input_csv = Path(f"{directory_path}/{f_name}_grouped.csv")
    df = pandas.read_csv(input_csv)
    pouet = df['object_ID_list']
    input_ROI = Path(f"{directory_path}/{f_name}.zip")
    rm.open(f"{input_ROI}")
    for i in range(len(pouet)):
        single_row = df.iloc[i]
        random_color = random.choice(colors)
        row_cellids = single_row.object_ID_list
        row_cleaned = row_cellids.strip('[').strip(']')
        row_array = row_cleaned.split(', ')
        for cell in row_array:
            cell_index = int(cell)
            roi = rm.select(cell_index)
            overlay_command = f"Overlay.addSelection('{random_color}',2);"
            ij.py.run_macro(overlay_command)
    ij.py.run_macro("setMinAndMax(0, 50);")
    ij.py.run_macro("run('Flatten', 'stack');")
    method = 'max all'
    z_projector_result = ZProjector.run(imp, method)
    z_collapsed_image = ij.py.from_java(z_projector_result)
    z_collapsed_dataset = ij.py.to_dataset(z_projector_result)
    result_path = os.path.splitext(file_path)[0] + "_overlay.tif"
    ij.io().save(z_collapsed_dataset, result_path)
    ij.py.run_macro("close();")
    ij.py.run_macro("roiManager('Select All');")
    rm.runCommand("Delete")
    

### Quantification on single MCV

In [ ]:
channel_path = Path(f"{directory_path}/other")
file_pattern = os.path.join(channel_path, "*.tif")
file_list = glob.glob(file_pattern)

In [ ]:
len_list = int(len(file_list)/2)
print(len_list)

In [ ]:
#set measurement 
set_string = f'Set Measurements...'
measure_string = f'mean stack redirect=None decimal=2'
ij.IJ.run(set_string, measure_string)


for i in range (1, len_list+1):
    # first channel of interest
    channellc3 = Path(f"{directory_path}/other/cell_test_bact{i}_LC3.tif")
    lc3_name = os.path.basename(channellc3)
    lc3 = f'open("{channellc3}")'
    
    # second channel of interest
    channellv = Path(f"{directory_path}/other/cell_test_bact{i}_LV.tif")
    lv_name = os.path.basename(channellv)
    lv = f'open("{channellv}")'
    
    
    # open ROI set and previous "grouped" csv file
    input_csv = Path(f"{directory_path}/cell_test_bact{i}_grouped.csv")
    input_roi = Path(f"{directory_path}/cell_test_bact{i}.zip")
    df0 = pandas.read_csv(input_csv)
    rm.open(f"{input_roi}")
    
    
    #measurement LC3
    image_lc3 = ij.py.run_macro(lc3)
    for n in range (len(df0)):
        single_row = df0['object_ID_list'][n] #testing.iloc[10] #row number -2
        row_cleaned = single_row.strip('[').strip(']')
        terms = row_cleaned.split(", ")
        nums = []
        for term in terms:
            nums.append(int(term))
        nums.sort()
        for bact in nums:
            roi = rm.select(bact)
            ij.IJ.run('Measure')
    output = Path(f"{directory_path}/{lc3_name}.csv").as_posix()
    saving = ij.IJ.saveAs("Results", output)
    ij.IJ.run("Clear Results")
    ij.IJ.selectWindow(f"{lc3_name}")
    ij.IJ.run('Close')
    #ij.IJ.selectWindow("Results")
    #ij.IJ.run('Close')

    #measurement LV
    image_lv = ij.py.run_macro(lv)
    for n in range (len(df0)):
        single_row = df0['object_ID_list'][n] #testing.iloc[10] #row number -2
        row_cleaned = single_row.strip('[').strip(']')
        terms = row_cleaned.split(", ")
        nums = []
        for term in terms:
            nums.append(int(term))
        nums.sort()
        for bact in nums:
            roi = rm.select(bact)
            ij.IJ.run('Measure')
    output = Path(f"{directory_path}/{lv_name}.csv").as_posix()
    saving = ij.IJ.saveAs("Results", output)
    ij.IJ.run("Clear Results")
    ij.IJ.selectWindow(f"{lv_name}")
    ij.IJ.run('Close')
    
    ij.py.run_macro("roiManager('Select All');")
    rm.runCommand("Delete")

In [ ]:
file_pattern = os.path.join(directory_path, "*.tif.csv")
file_list = glob.glob(file_pattern)
len_list = int(len(file_list)/2)
print(len_list)

In [ ]:
for i in range (1, len_list+1):
    input_csv_lc3 = Path(f"{directory_path}/cell_test_bact{i}_LC3.tif.csv")
    input_csv_lv = Path(f"{directory_path}/cell_test_bact{i}_LV.tif.csv")
    #first marker csv
    df1 = pandas.read_csv(input_csv_lc3)
    df1.rename(columns={'Mean': 'Mean_LC3'}, inplace=True)
    df1 = df1.drop(columns = [df1.columns[0],df1.columns[2]], axis =1)

    #second marker csv
    df2 = pandas.read_csv(input_csv_lv)
    df2.rename(columns={'Mean': 'Mean_LV'}, inplace=True)
    df2 = df2.drop(columns =[df2.columns[0],df2.columns[2], df2.columns[3]], axis =1)

    #create new merged dataframe:
    final_results = pandas.concat([df1, df2], axis = 1)
    final_results = final_results.iloc[:, [1, 0, 2]]
    out = directory_path +"/final_output/"
    if not os.path.exists(out):
        os.makedirs(out)
    output_path = Path(f"{out}/cell_test_bact{i}_final_result.csv")
    final_results.to_csv(output_path)